In [6]:
import pandas as pd
import numpy as np
import sys, os

In [7]:
# data = pd.read_excel("data/solarsummaries.xlsx")
# data.head()
# xl = pd.ExcelFile("data/solarsummaries.xlsx")
# print(xl.sheet_names) # display a list of the sheets in the spreadsheet

In [19]:
# Define the lat, long of the location and the year
lat, lon, year = 33.2164, -97.1292, 2010
# You must request an NSRDB api key from the link above
api_key = 'laKb0T0Fc9h4lKYiGVSQklvUNfrtWyld83y1k6Ne'
# Set the attributes to extract (e.g., dhi, ghi, etc.), separated by commas.
attributes = 'ghi,dhi,dni,wind_speed_10m_nwp,surface_air_temperature_nwp,solar_zenith_angle'
# Choose year of data
year = '2010'
# Set leap year to true or false. True will return leap day data if present, false will not.
leap_year = 'false'
# Set time interval in minutes, i.e., '30' is half hour intervals. Valid intervals are 30 & 60.
interval = '30'
# Specify Coordinated Universal Time (UTC), 'true' will use UTC, 'false' will use the local time zone of the data.
# NOTE: In order to use the NSRDB data in SAM, you must specify UTC as 'false'. SAM requires the data to be in the
# local time zone.
utc = 'false'
# Your full name, use '+' instead of spaces.
your_name = 'Jack+Sullivan'
# Your reason for using the NSRDB.
reason_for_use = 'beta+testing'
# Your affiliation
your_affiliation = 'UC+Berkeley'
# Your email address
your_email = 'jsullivan27@berkeley.edu'
# Please join our mailing list so we can keep you up-to-date on new developments.
mailing_list = 'false'

# Declare url string
url = 'http://developer.nrel.gov/api/solar/nsrdb_0512_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api_key, attr=attributes)

In [24]:
# Return just the first 2 lines to get metadata:
df = pd.read_csv('http://developer.nrel.gov/api/solar/nsrdb_0512_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api_key, attr=attributes), skiprows=2)
# See metadata for specified properties, e.g., timezone and elevation
#timezone, elevation = info['Local Time Zone'], info['Elevation']

In [27]:
df.head()

,Year,Month,Day,Hour,Minute,GHI,DHI,DNI,Wind Speed,Temperature,Solar Zenith Angle
0,2010,1,1,0,0,0,0,0,3.594724,-2.604285,167.605189
1,2010,1,1,0,30,0,0,0,3.414035,-2.867499,169.785147
2,2010,1,1,1,0,0,0,0,3.233346,-3.130713,168.080732
3,2010,1,1,1,30,0,0,0,3.114784,-3.304114,163.670063
4,2010,1,1,2,0,0,0,0,2.996223,-3.477515,158.144707


In [28]:
df.columns.values

array(['Year', 'Month', 'Day', 'Hour', 'Minute', 'GHI', 'DHI', 'DNI',
       'Wind Speed', 'Temperature', 'Solar Zenith Angle'], dtype=object)